# goal of this notebook

Before we can use the data we have to prepare it in consumable structure where *period* and *region* are row indices and all other columns are variables (including target ones)

All missing data will be filled with ffill()

In [1]:
import pandas as pd
from functools import reduce

In [2]:
df = pd.read_excel(r"numero_uno_11.xlsx")
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,"Уровень инновационной активности, в процентах (%)",Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,"Индекс производительности труда по Российской Федерации, \nпо субъектам Российской Федерации в 2008-2022 г.г. (в % к предыдущему году)",Unnamed: 9,...,11411000200010200019 Объем инвестиций в основной капитал (за исключением бюджетных средств) с 2018 г.(тыс. руб.),Unnamed: 419,Unnamed: 420,Unnamed: 421,Unnamed: 422,11111110200300200001 Текущая рыночная стоимость жилых зданий(млн. руб.),Unnamed: 424,Unnamed: 425,Unnamed: 426,Unnamed: 427
0,Index,Index,region_name,2018.000000,2019.0,2020.000000,2021.000000,2022.000000,2018.0,2019.0,...,2018 г.,2019 г.,2020 г.,2021 г.,2022 г.,2018 г.,2019 г.,2020 г.,2021 г.,2022 г.
1,1,14000000000,Белгородская область,18.151815,15.1,18.011895,16.998012,15.130941,102.8,102.2,...,122692316,147140843,148193081,"150464999,99","166351412,99","1305087,99","1343136,99","1383018,99","1531325,99","1782291,99"
2,2,15000000000,Брянская область,8.243243,10.1,10.943396,13.517442,12.094395,104.9,105.0,...,"51322637,99","52841525,99",61748381,71453729,75388303,"727575,99","715541,99","733878,99","750697,99","774654,99"
3,3,17000000000,Владимирская область,13.006135,10.5,12.648221,11.666667,9.867173,100.8,104.0,...,64891800,"77123595,99",79480131,"71849385,99",82673964,"927909,99","959045,99","933258,99","964891,99","1325435,99"
4,4,20000000000,Воронежская область,17.097416,13.4,15.937500,12.626656,10.436714,102.7,102.0,...,"237032908,99",248907057,"208205274,99","239142522,99",273153509,"1364237,99","1366961,99","1472497,99","1753907,99","2421244,99"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.columns.tolist()

['Unnamed: 0',
 'Unnamed: 1',
 'Unnamed: 2',
 'Уровень инновационной активности, в процентах (%)',
 'Unnamed: 4',
 'Unnamed: 5',
 'Unnamed: 6',
 'Unnamed: 7',
 'Индекс производительности труда по Российской Федерации, \nпо субъектам Российской Федерации в 2008-2022 г.г. (в % к предыдущему году)',
 'Unnamed: 9',
 'Unnamed: 10',
 'Unnamed: 11',
 'Unnamed: 12',
 'Уровень бедности, в процентах (%) от общей численности населения',
 'Unnamed: 14',
 'Unnamed: 15',
 'Unnamed: 16',
 'Unnamed: 17',
 'Индекс промышленного производства в % к соответствующему периоду предыдущего года',
 'Unnamed: 19',
 'Unnamed: 20',
 'Unnamed: 21',
 'Unnamed: 22',
 'Распределение объема работ, выполненных по виду деятельности "Строительство" организациями различных форм собственности \n(без учета субъектов малого предпринимательства) \n(тыс. рублей)',
 'Unnamed: 24',
 'Unnamed: 25',
 'Unnamed: 26',
 'Unnamed: 27',
 'Среднедушевые денежные доходы населения по субъектам Российской Федерации, руб./месяц',
 'Unnamed: 

In [4]:
def split_list_with_condition(lst):
    result = []
    temp = []
    for i in range(len(lst)):
        if 'Unnamed' in lst[i] and (i == len(lst) - 1 or 'Unnamed' not in lst[i+1]):
            temp.append(lst[i])
            if temp:
                result.append(temp)
                temp = []
        else:
            temp.append(lst[i])
    return result

blocks = split_list_with_condition(df.columns.tolist())[1:]
blocks

[['Уровень инновационной активности, в процентах (%)',
  'Unnamed: 4',
  'Unnamed: 5',
  'Unnamed: 6',
  'Unnamed: 7'],
 ['Индекс производительности труда по Российской Федерации, \nпо субъектам Российской Федерации в 2008-2022 г.г. (в % к предыдущему году)',
  'Unnamed: 9',
  'Unnamed: 10',
  'Unnamed: 11',
  'Unnamed: 12'],
 ['Уровень бедности, в процентах (%) от общей численности населения',
  'Unnamed: 14',
  'Unnamed: 15',
  'Unnamed: 16',
  'Unnamed: 17'],
 ['Индекс промышленного производства в % к соответствующему периоду предыдущего года',
  'Unnamed: 19',
  'Unnamed: 20',
  'Unnamed: 21',
  'Unnamed: 22'],
 ['Распределение объема работ, выполненных по виду деятельности "Строительство" организациями различных форм собственности \n(без учета субъектов малого предпринимательства) \n(тыс. рублей)',
  'Unnamed: 24',
  'Unnamed: 25',
  'Unnamed: 26',
  'Unnamed: 27'],
 ['Среднедушевые денежные доходы населения по субъектам Российской Федерации, руб./месяц',
  'Unnamed: 29',
  'Unnam

In [5]:
true_blocks = [col for col in blocks if len(col) == 5]
true_blocks

[['Уровень инновационной активности, в процентах (%)',
  'Unnamed: 4',
  'Unnamed: 5',
  'Unnamed: 6',
  'Unnamed: 7'],
 ['Индекс производительности труда по Российской Федерации, \nпо субъектам Российской Федерации в 2008-2022 г.г. (в % к предыдущему году)',
  'Unnamed: 9',
  'Unnamed: 10',
  'Unnamed: 11',
  'Unnamed: 12'],
 ['Уровень бедности, в процентах (%) от общей численности населения',
  'Unnamed: 14',
  'Unnamed: 15',
  'Unnamed: 16',
  'Unnamed: 17'],
 ['Индекс промышленного производства в % к соответствующему периоду предыдущего года',
  'Unnamed: 19',
  'Unnamed: 20',
  'Unnamed: 21',
  'Unnamed: 22'],
 ['Распределение объема работ, выполненных по виду деятельности "Строительство" организациями различных форм собственности \n(без учета субъектов малого предпринимательства) \n(тыс. рублей)',
  'Unnamed: 24',
  'Unnamed: 25',
  'Unnamed: 26',
  'Unnamed: 27'],
 ['Среднедушевые денежные доходы населения по субъектам Российской Федерации, руб./месяц',
  'Unnamed: 29',
  'Unnam

In [6]:
index_regions = df[df.columns[2]].values[1:]
index_regions

array(['Белгородская область', 'Брянская область', 'Владимирская область',
       'Воронежская область', 'Ивановская область', 'Калужская область',
       'Костромская область', 'Курская область', 'Липецкая область',
       'Московская область', 'Орловская область', 'Рязанская область',
       'Смоленская область', 'Тамбовская область', 'Тверская область',
       'Тульская область', 'Ярославская область',
       'Город Москва столица Российской Федерации город федерального значения',
       'Республика Карелия', 'Республика Коми',
       'Ненецкий автономный округ (Архангельская область)',
       'Архангельская область (кроме Ненецкого автономного округа)',
       'Вологодская область', 'Калининградская область',
       'Ленинградская область', 'Мурманская область',
       'Новгородская область', 'Псковская область',
       'Город Санкт-Петербург город федерального значения',
       'Республика Адыгея (Адыгея)', 'Республика Калмыкия',
       'Республика Крым', 'Краснодарский край', 'Ас

In [7]:
real_df = []
index_names = []
for col in true_blocks:
    name = df[col[0]].name
    unit = df[col]
    unit.columns = unit.iloc[0]
    unit = unit.drop(unit.index[0])
    unit.set_index(index_regions, inplace=True)
    unit.columns = [x for x in range(2018, 2023, 1)]
    unit = unit.reset_index()
    unit = unit.dropna(subset=['index'])
    real_df.append(unit)
    index_names.append(name)
real_df[0]

,index,2018,2019,2020,2021,2022
0,Белгородская область,18.151815,15.1,18.011895,16.998012,15.130941
1,Брянская область,8.243243,10.1,10.943396,13.517442,12.094395
2,Владимирская область,13.006135,10.5,12.648221,11.666667,9.867173
3,Воронежская область,17.097416,13.4,15.937500,12.626656,10.436714
4,Ивановская область,8.133971,10.0,16.196137,14.893617,13.756614
...,...,...,...,...,...,...
80,Амурская область,6.250000,5.1,6.657790,5.947441,6.824513
81,Магаданская область,10.326087,13.3,9.427609,9.121622,8.552632
82,Сахалинская область,5.982906,5.0,4.809052,3.896104,4.848485
83,Еврейская автономная область,7.246377,4.3,5.500000,4.494382,3.954802


In [8]:
mrlted_ls = []

for col, name, unit in zip(true_blocks, index_names, real_df):
    melted_df = unit.melt(id_vars='index', var_name='period', value_name=name)
    mrlted_ls.append(melted_df)

In [9]:
merged_df = mrlted_ls[0]

for frame in mrlted_ls[1:]:
    merged_df = pd.concat([merged_df, frame[frame.columns[-1]]], axis=1)
merged_df

,index,period,"Уровень инновационной активности, в процентах (%)","Индекс производительности труда по Российской Федерации, \nпо субъектам Российской Федерации в 2008-2022 г.г. (в % к предыдущему году)","Уровень бедности, в процентах (%) от общей численности населения",Индекс промышленного производства в % к соответствующему периоду предыдущего года,"Распределение объема работ, выполненных по виду деятельности ""Строительство"" организациями различных форм собственности \n(без учета субъектов малого предпринимательства) \n(тыс. рублей)","Среднедушевые денежные доходы населения по субъектам Российской Федерации, руб./месяц","Численность занятых в возрасте 15 лет и старше по субъектам Российской Федерации (по данным выборочных обследований), тыс. человек","Уровень занятости населения в возрасте 15 лет и старше по субъектам Российской Федерации (по данным выборочных обследований), в процентах",...,Прирост высокопроизводительных рабочих мест по субъектам Российской Федерации за 2012-2022 гг. (в %),11111150100320200001 Плотность автомобильных дорог с твердым покрытием общего пользования на 1000 кв. км территории (по 2013 г.),22720000300200200001 Общая численность пенсионеров,"13230000100060200002 Численность врачей всех специальностей (физических лиц) в учреждениях, оказывающих медицинские услуги населению, на конец отчетного года(тыс. чел.)Терапевты","13230000100060200002 Численность врачей всех специальностей (физических лиц) в учреждениях, оказывающих медицинские услуги населению, на конец отчетного года(тыс. чел.)Всего врачей",22310000100520200001 Количество введенных жилых помещений ( квартир ) в расчете на 1000 человек населения,"22310000200010200001 Средняя факт. стоимость строительства 1 кв.м. общей площади отдельно стоящих жилых домов без пристроек, надстроек и встроенных помещений, Отдельно стоящие здания жилого назначения квартирного типа",22310000100030200001 Введено в действие общей площади жилых домов на 1000 человек населения (годовая),11411000200010200019 Объем инвестиций в основной капитал (за исключением бюджетных средств) с 2018 г.(тыс. руб.),11111110200300200001 Текущая рыночная стоимость жилых зданий(млн. руб.)
0,Белгородская область,2018,18.151815,102.8,7.5,102.4,23362304.1,30712.962414,792.637,60.6,...,"16,7",731,539,"0,46","6,26","7,70",47772,785,122692316,"1305087,99"
1,Брянская область,2018,8.243243,104.9,13.6,102.7,5311814.7,26722.337614,586.497,57.2,...,"10,6",320,439,"0,59","4,86","5,09",30541,334,"51322637,99","727575,99"
2,Владимирская область,2018,13.006135,100.8,13.1,104.6,5663657.5,23343.519683,688.375,58.7,...,"15,0",347,489,"0,62","4,76","5,50","31854,99",476,64891800,"927909,99"
3,Воронежская область,2018,17.097416,102.7,8.9,103.2,29882520.9,30158.281326,1141.811,57.3,...,"19,9",359,"797,99","1,06","12,16","9,40",32583,726,"237032908,99","1364237,99"
4,Ивановская область,2018,8.133971,102.3,14.2,102.1,4499339.9,25421.329962,502.984,58.2,...,"15,7",334,"346,99","0,56","4,44","4,50",29309,366,26071299,"500732,99"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,Амурская область,2022,6.824513,102.6,11.9,102.8,158807105.1,49798.000000,382.853,61.1,...,"2,4",35,238,"1,07","9,56","6,90",53775,498,"445380621,99","125263,99"
421,Магаданская область,2022,8.552632,95.9,6.7,101.1,15242720.8,101999.000000,80.746,71.2,...,"4,9","5,60","45,99","0,78","7,09","1,40","188901,99",85,"67783415,99","429248,99"
422,Сахалинская область,2022,4.848485,93.9,6.3,76.8,68785826.1,79322.000000,261.753,67.1,...,"16,5",33,"168,99","0,54","4,27","15,40",NaN,1088,"234063071,99","94554,99"
423,Еврейская автономная область,2022,3.954802,98.4,18.3,107.0,11158163.2,37753.000000,72.355,58.6,...,"-3,3",71,"47,99","0,09","0,84","1,70",NaN,170,"18087344,99","16654,99"


In [10]:
merged_df.to_excel('panel.xlsx', index=False)